# 🚀 Beginner's Guide to Agent-to-Agent (A2A) Hiring Assistant

## Welcome to AI-Powered Recruitment!

This notebook demonstrates how two simple AI agents can work together to evaluate job candidates:
- **📊 Rating Agent**: Evaluates technical skills and cultural fit
- **🔍 Background Check Agent**: Verifies education, employment, and achievements

### What You'll Learn:
1. How to set up AI agents with tool calling capabilities
2. How agents can communicate and work together
3. How to build an inclusive, bias-aware hiring system

### Prerequisites:
- Azure OpenAI API key and endpoint
- **No coding experience required!** We'll walk you through everything step by step

**Don't have Python installed?** No problem! This guide covers everything from scratch.

Let's get started! 🎯

## 📦 Step 1: Create Virtual Environment

Just click run on the cell below to create a Python virtual environment.

In [25]:
!python3 -m venv light_demo_venv

## 📦 Step 2: Install Required Packages

Just click run on the cell below to install all packages needed for this notebook.

In [ ]:
!source light_demo_venv/bin/activate && pip install openai python-dotenv pydantic typing-extensions ipykernel

  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached anyio-4.10.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.10.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached httpcore-1.0.9-py3-none

## ⚠️ Important: Select Virtual Environment in VS Code

After running the cells above, you need to select the virtual environment as your Python interpreter:

**In VS Code:**
1. **Press `Cmd+Shift+P`** (or `Ctrl+Shift+P` on Windows/Linux)
2. **Type "Python: Select Interpreter"**
3. **Select the virtual environment:** `./light_demo_venv/bin/python`

**OR click the Python version in the top-right corner** of VS Code and select the virtual environment.

Now all cells will run in the virtual environment with the packages installed!

## 🔑 Step 3: Setup API Keys

Create a `.env` file in your project directory with your Azure OpenAI credentials, or enter them below:

### Option 1: Create .env file (Recommended)
1. Create a new file called `.env` in the same folder as this notebook
2. Add these lines (replace with your actual values):
```
AZURE_OPENAI_TOKEN=your_token_here
AZURE_OPENAI_ENDPOINT=your_endpoint_here
```

In [2]:
import os
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()

# If you don't have a .env file, uncomment and fill these in:
# os.environ["AZURE_OPENAI_TOKEN"] = "your_token_here"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "your_endpoint_here"

# Verify credentials are loaded
if not os.getenv("AZURE_OPENAI_TOKEN") or not os.getenv("AZURE_OPENAI_ENDPOINT"):
    print("⚠️ Please set your Azure OpenAI credentials!")
else:
    print("✅ Credentials loaded successfully!")

✅ Credentials loaded successfully!


In [3]:
from openai import AzureOpenAI
from typing import Dict, List, Any

class RatingAgent:
    """Simple rating agent for candidate evaluation."""
    
    def __init__(self):
        self.client = AzureOpenAI(
            api_key=os.getenv("AZURE_OPENAI_TOKEN"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_version="2025-03-01-preview"
        )
        
        # System message for technical evaluation
        self.tech_system_message = """
        You are a technical evaluator for SAP AI Scientist positions. Rate technical skills 1-10.
        
        LOOK FOR:
        ✓ Concrete projects with measurable impact
        ✓ Code examples or GitHub contributions
        ✓ Problem-solving examples
        ✓ Teaching/mentoring others (shows mastery)
        
        AVOID PENALIZING:
        ✗ Non-traditional education paths
        ✗ Career gaps (often due to caregiving)
        ✗ Non-native English speakers
        ✗ Different communication styles
        
        Focus ONLY on technical ability. Provide a score 1-10 and brief reasoning.
        """
        
        # System message for inclusion evaluation
        self.inclusion_system_message = """
        You are an inclusion evaluator for diverse SAP teams. Rate inclusion potential 1-10.
        
        LOOK FOR:
        ✓ Mentored underrepresented people
        ✓ Led diversity initiatives
        ✓ Cross-cultural collaboration experience
        ✓ Community involvement
        ✓ Creating inclusive environments
        
        BIAS AWARENESS - DO NOT PENALIZE:
        - Career gaps (often caregiving responsibilities)
        - Non-native English speakers who communicate clearly
        - Different cultural backgrounds
        - Age differences
        - Non-traditional career paths
        
        Base score starts at 7/10. Add points for demonstrated inclusive behaviors.
        Provide a score 1-10 and brief reasoning.
        """
    
    def evaluate_technical_skills(self, candidate_info: str) -> Dict[str, Any]:
        """Evaluate candidate's technical skills."""
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": self.tech_system_message},
                    {"role": "user", "content": f"Evaluate this candidate's technical skills:\n{candidate_info}"}
                ],
                temperature=0.3
            )
            
            result = response.choices[0].message.content
            
            # Extract score (simple parsing - in production, use structured output)
            score = self._extract_score(result)
            
            return {
                "score": score,
                "reasoning": result,
                "category": "Technical Skills"
            }
        except Exception as e:
            return {
                "score": 0,
                "reasoning": f"Error: {str(e)}",
                "category": "Technical Skills"
            }
    
    def evaluate_inclusion_potential(self, candidate_info: str) -> Dict[str, Any]:
        """Evaluate candidate's inclusion potential."""
        try:
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": self.inclusion_system_message},
                    {"role": "user", "content": f"Evaluate this candidate's inclusion potential:\n{candidate_info}"}
                ],
                temperature=0.3
            )
            
            result = response.choices[0].message.content
            score = self._extract_score(result)
            
            return {
                "score": score,
                "reasoning": result,
                "category": "Inclusion Potential"
            }
        except Exception as e:
            return {
                "score": 7,  # Default to neutral for inclusion
                "reasoning": f"Error: {str(e)}",
                "category": "Inclusion Potential"
            }
    
    def _extract_score(self, text: str) -> int:
        """Extract numeric score from response text."""
        import re
        
        # Look for patterns like "8/10", "Score: 7", etc.
        patterns = [
            r'(\d+)/10',
            r'Score:?\s*(\d+)',
            r'Rating:?\s*(\d+)',
            r'(\d+)\s*out\s*of\s*10'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                score = int(match.group(1))
                return max(1, min(10, score))  # Ensure score is between 1-10
        
        # If no score found, return neutral
        return 5

# Create our rating agent
rating_agent = RatingAgent()
print("✅ Rating Agent created successfully!")

✅ Rating Agent created successfully!


## 🔍 Step 5: Create Our Background Check Agent

This agent verifies educational credentials, work history, and project claims using tool calling.

In [4]:
class BackgroundCheckAgent:
    """Simple background verification agent with tool calling."""
    
    def __init__(self):
        self.client = AzureOpenAI(
            api_key=os.getenv("AZURE_OPENAI_TOKEN"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_version="2025-03-01-preview"
        )
        
        # Define verification tools
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "verify_university",
                    "description": "Verify if a university is legitimate and accredited",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "university_name": {
                                "type": "string",
                                "description": "Name of the university to verify"
                            }
                        },
                        "required": ["university_name"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "verify_company",
                    "description": "Verify if a company exists and is legitimate",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "company_name": {
                                "type": "string",
                                "description": "Name of the company to verify"
                            }
                        },
                        "required": ["company_name"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "verify_project",
                    "description": "Verify if a project or achievement is plausible",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "project_description": {
                                "type": "string",
                                "description": "Description of the project to verify"
                            }
                        },
                        "required": ["project_description"]
                    }
                }
            }
        ]
        
        self.system_message = """
        You are a background verification specialist. When analyzing candidate information:
        
        1. Extract all universities, companies, and significant projects mentioned
        2. Use the verification tools to check each item
        3. Provide a comprehensive background check report
        4. Give overall status: VERIFIED, PARTIALLY VERIFIED, or NOT VERIFIED
        
        Always use the verification tools before making conclusions.
        """
    
    def verify_university(self, university_name: str) -> str:
        """Verify university legitimacy - simplified version for demo."""
        legitimate_universities = {
            "harvard", "stanford", "mit", "berkeley", "tsinghua", "peking", 
            "nus", "ntu", "university of tokyo", "eth zurich", "cambridge", "oxford",
            "cmu", "caltech", "princeton", "yale", "columbia", "chicago", "cornell"
        }
        
        university_lower = university_name.lower()
        is_legitimate = any(known in university_lower for known in legitimate_universities)
        
        return f"University '{university_name}' verification: {'VERIFIED' if is_legitimate else 'NOT VERIFIED'}"
    
    def verify_company(self, company_name: str) -> str:
        """Verify company legitimacy - simplified version for demo."""
        legitimate_companies = {
            "google", "microsoft", "apple", "amazon", "meta", "netflix", "tesla",
            "sap", "oracle", "ibm", "salesforce", "adobe", "nvidia", "intel",
            "alibaba", "tencent", "baidu", "bytedance", "huawei", "xiaomi"
        }
        
        company_lower = company_name.lower()
        is_legitimate = any(known in company_lower for known in legitimate_companies)
        
        return f"Company '{company_name}' verification: {'VERIFIED' if is_legitimate else 'NOT VERIFIED'}"
    
    def verify_project(self, project_description: str) -> str:
        """Verify project plausibility - simplified version for demo."""
        suspicious_keywords = {
            "world champion", "nobel prize", "invented", "discovered", 
            "revolutionary breakthrough", "solved climate change", "cured cancer"
        }
        
        project_lower = project_description.lower()
        is_suspicious = any(keyword in project_lower for keyword in suspicious_keywords)
        
        return f"Project verification: {'NEEDS FURTHER REVIEW' if is_suspicious else 'PLAUSIBLE'}"
    
    def perform_background_check(self, candidate_info: str) -> Dict[str, Any]:
        """Perform comprehensive background check."""
        try:
            # Call the AI with tools
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": self.system_message},
                    {"role": "user", "content": f"Perform background check on: {candidate_info}"}
                ],
                tools=self.tools,
                tool_choice="auto",
                temperature=0.1
            )
            
            # Handle tool calls
            if response.choices[0].message.tool_calls:
                return self._handle_tool_calls(response, candidate_info)
            else:
                return {
                    "verification_status": "NO_VERIFICATION_NEEDED",
                    "details": response.choices[0].message.content,
                    "tool_results": []
                }
                
        except Exception as e:
            return {
                "verification_status": "ERROR",
                "details": f"Background check failed: {str(e)}",
                "tool_results": []
            }
    
    def _handle_tool_calls(self, response, original_query: str) -> Dict[str, Any]:
        """Handle tool function calls."""
        messages = [
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": f"Perform background check on: {original_query}"},
            response.choices[0].message
        ]
        
        tool_results = []
        
        # Execute each tool call
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            # Call the appropriate function
            if function_name == "verify_university":
                result = self.verify_university(function_args["university_name"])
            elif function_name == "verify_company":
                result = self.verify_company(function_args["company_name"])
            elif function_name == "verify_project":
                result = self.verify_project(function_args["project_description"])
            else:
                result = "Unknown function"
            
            tool_results.append({
                "function": function_name,
                "args": function_args,
                "result": result
            })
            
            # Add function result to messages
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": result
            })
        
        # Get final response with tool results
        final_response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0.1
        )
        
        final_content = final_response.choices[0].message.content
        
        # Determine verification status
        if "NOT VERIFIED" in str(tool_results):
            status = "PARTIALLY_VERIFIED"
        elif "NEEDS FURTHER REVIEW" in str(tool_results):
            status = "NEEDS_REVIEW"
        else:
            status = "VERIFIED"
        
        return {
            "verification_status": status,
            "details": final_content,
            "tool_results": tool_results
        }

# Create our background check agent
background_agent = BackgroundCheckAgent()
print("✅ Background Check Agent created successfully!")

✅ Background Check Agent created successfully!


## 🤝 Step 6: Create the Agent Coordinator

This coordinates the two agents to work together and combines their results.

In [5]:
class A2ACoordinator:
    """Coordinates the rating and background check agents."""
    
    def __init__(self, rating_agent: RatingAgent, background_agent: BackgroundCheckAgent):
        self.rating_agent = rating_agent
        self.background_agent = background_agent
    
    def evaluate_candidate(self, candidate_info: str) -> Dict[str, Any]:
        """Run complete candidate evaluation using both agents."""
        print("🚀 Starting candidate evaluation...")
        
        # Step 1: Rating Agent evaluates technical skills and inclusion potential
        print("📊 Rating Agent: Evaluating technical skills...")
        tech_evaluation = self.rating_agent.evaluate_technical_skills(candidate_info)
        
        print("🤝 Rating Agent: Evaluating inclusion potential...")
        inclusion_evaluation = self.rating_agent.evaluate_inclusion_potential(candidate_info)
        
        # Step 2: Background Check Agent verifies credentials
        print("🔍 Background Check Agent: Verifying credentials...")
        background_check = self.background_agent.perform_background_check(candidate_info)
        
        # Step 3: Combine results
        print("✅ Combining results...")
        
        overall_score = (tech_evaluation["score"] + inclusion_evaluation["score"]) / 2
        
        # Adjust overall score based on background check
        if background_check["verification_status"] == "NOT_VERIFIED":
            overall_score *= 0.5  # Significant penalty for unverified info
        elif background_check["verification_status"] == "NEEDS_REVIEW":
            overall_score *= 0.8  # Moderate penalty for suspicious claims
        
        return {
            "candidate_info": candidate_info,
            "technical_evaluation": tech_evaluation,
            "inclusion_evaluation": inclusion_evaluation,
            "background_check": background_check,
            "overall_score": round(overall_score, 1),
            "recommendation": self._get_recommendation(overall_score, background_check["verification_status"])
        }
    
    def _get_recommendation(self, score: float, verification_status: str) -> str:
        """Generate hiring recommendation based on score and verification."""
        if verification_status == "NOT_VERIFIED":
            return "❌ NOT RECOMMENDED - Unverified credentials"
        elif score >= 8.0:
            return "🌟 HIGHLY RECOMMENDED - Strong candidate"
        elif score >= 6.0:
            return "👍 RECOMMENDED - Good candidate"
        elif score >= 4.0:
            return "🤔 CONSIDER - Potential with development"
        else:
            return "❌ NOT RECOMMENDED - Below threshold"
    
    def print_evaluation_report(self, evaluation: Dict[str, Any]):
        """Print a nicely formatted evaluation report."""
        print("\n" + "="*60)
        print("🎯 CANDIDATE EVALUATION REPORT")
        print("="*60)
        
        print(f"\n📊 TECHNICAL SKILLS: {evaluation['technical_evaluation']['score']}/10")
        print(f"Reasoning: {evaluation['technical_evaluation']['reasoning'][:200]}...")
        
        print(f"\n🤝 INCLUSION POTENTIAL: {evaluation['inclusion_evaluation']['score']}/10")
        print(f"Reasoning: {evaluation['inclusion_evaluation']['reasoning'][:200]}...")
        
        print(f"\n🔍 BACKGROUND CHECK: {evaluation['background_check']['verification_status']}")
        if evaluation['background_check']['tool_results']:
            print("Verification Details:")
            for result in evaluation['background_check']['tool_results']:
                print(f"  • {result['function']}: {result['result']}")
        
        print(f"\n🎯 OVERALL SCORE: {evaluation['overall_score']}/10")
        print(f"📝 RECOMMENDATION: {evaluation['recommendation']}")
        print("\n" + "="*60)

# Create our coordinator
coordinator = A2ACoordinator(rating_agent, background_agent)
print("✅ A2A Coordinator created successfully!")
print("\n🎉 All agents are ready! Let's evaluate some candidates!")

✅ A2A Coordinator created successfully!

🎉 All agents are ready! Let's evaluate some candidates!


## 🧪 Step 7: Test with Sample Candidates

Let's test our A2A system with different candidate profiles to see how it works!

### Candidate 1: Strong Technical Background

In [6]:
candidate_1 = """
Name: Sarah Chen
Education: PhD Computer Science, Stanford University (2020)
Experience: 
- Senior AI Engineer at Google (2020-2024)
- Built transformer models for Google Translate, improved accuracy by 23%
- Led team of 8 engineers, delivered 3 production ML systems
- Open source contributor: 2.5k GitHub stars on ML optimization library

Projects:
- Designed distributed training pipeline handling 100M+ parameters
- Optimized inference latency from 500ms to 50ms for real-time application
- Published 5 papers in top-tier ML conferences (NeurIPS, ICML)

Leadership & Inclusion:
- Mentored 12 junior engineers, 6 were women or underrepresented minorities
- Started "AI for Social Good" initiative, helped 3 nonprofits build ML solutions
- Volunteer coding instructor for refugee resettlement program
- Led unconscious bias training for 50+ engineers
"""

print("Evaluating Candidate 1: Sarah Chen")
evaluation_1 = coordinator.evaluate_candidate(candidate_1)
coordinator.print_evaluation_report(evaluation_1)

Evaluating Candidate 1: Sarah Chen
🚀 Starting candidate evaluation...
📊 Rating Agent: Evaluating technical skills...
🤝 Rating Agent: Evaluating inclusion potential...
🔍 Background Check Agent: Verifying credentials...
✅ Combining results...

🎯 CANDIDATE EVALUATION REPORT

📊 TECHNICAL SKILLS: 10/10
Reasoning: **Score: 10/10**

**Reasoning:**
Sarah Chen demonstrates exceptional technical skills and impact across multiple dimensions:

1. **Concrete Projects with Measurable Impact**: 
   - Improved Google Tra...

🤝 INCLUSION POTENTIAL: 10/10
Reasoning: **Score: 10/10**

**Reasoning:**
Sarah Chen demonstrates exceptional inclusion potential through her leadership and community involvement. She has mentored a significant number of underrepresented ind...

🔍 BACKGROUND CHECK: VERIFIED
Verification Details:
  • verify_university: University 'Stanford University' verification: VERIFIED
  • verify_company: Company 'Google' verification: VERIFIED
  • verify_project: Project verification: PLAUSIBL

### Candidate 2: Non-Traditional Background

In [7]:
candidate_2 = """
Name: Marcus Rodriguez
Education: Self-taught programmer, Completed online bootcamp (2019)
Experience:
- Started as Junior Developer at local startup (2019)
- Promoted to Lead Developer after building customer analytics platform
- Freelancer for 2 years (2021-2023) - built 15+ web applications
- Currently Senior Developer at Microsoft (2023-present)

Projects:
- Built recommendation engine that increased user engagement by 40%
- Created automated testing framework reducing bug reports by 60%
- Developed mobile app with 10k+ active users

Background:
- Career gap 2016-2019 (caring for sick parent)
- First-generation college student, switched to tech from retail management
- Active in local tech meetups, organizes "Coding for Career Changers" workshop
- Bilingual (Spanish/English), helps translate tech documentation for Latino community
"""

print("\nEvaluating Candidate 2: Marcus Rodriguez")
evaluation_2 = coordinator.evaluate_candidate(candidate_2)
coordinator.print_evaluation_report(evaluation_2)


Evaluating Candidate 2: Marcus Rodriguez
🚀 Starting candidate evaluation...
📊 Rating Agent: Evaluating technical skills...
🤝 Rating Agent: Evaluating inclusion potential...
🔍 Background Check Agent: Verifying credentials...
✅ Combining results...

🎯 CANDIDATE EVALUATION REPORT

📊 TECHNICAL SKILLS: 9/10
Reasoning: **Score: 9/10**

**Reasoning:**
Marcus Rodriguez demonstrates strong technical skills through a combination of impactful projects, career progression, and community contributions. Here's the breakdown...

🤝 INCLUSION POTENTIAL: 9/10
Reasoning: **Score: 9/10**

**Reasoning:**

Marcus demonstrates strong inclusion potential based on his background and actions:

- **Mentored underrepresented people:** Organizing "Coding for Career Changers" wo...

🔍 BACKGROUND CHECK: PARTIALLY_VERIFIED
Verification Details:
  • verify_company: Company 'local startup' verification: NOT VERIFIED
  • verify_company: Company 'Microsoft' verification: VERIFIED
  • verify_project: Project verification

### Candidate 3: Suspicious Claims

In [8]:
candidate_3 = """
Name: Alex Thompson
Education: PhD from International University of Advanced Sciences (2018)
Experience:
- Chief AI Scientist at TechnoFuture Corp (2018-2024)
- "Invented revolutionary AI algorithm that solved climate change"
- "Won Nobel Prize consideration for breakthrough in quantum computing"
- "Published 50+ papers in top journals"

Projects:
- "Created AGI system with 99.9% accuracy on all tasks"
- "Discovered new mathematical theorem for AI optimization"
- "Built system that predicts stock market with 95% accuracy"

Skills:
- "Expert in all programming languages"
- "Fluent in 15 languages"
- "IQ of 200, certified genius"
"""

print("\nEvaluating Candidate 3: Alex Thompson")
evaluation_3 = coordinator.evaluate_candidate(candidate_3)
coordinator.print_evaluation_report(evaluation_3)


Evaluating Candidate 3: Alex Thompson
🚀 Starting candidate evaluation...
📊 Rating Agent: Evaluating technical skills...
🤝 Rating Agent: Evaluating inclusion potential...
🔍 Background Check Agent: Verifying credentials...
✅ Combining results...

🎯 CANDIDATE EVALUATION REPORT

📊 TECHNICAL SKILLS: 3/10
Reasoning: Score: 3/10

Reasoning:
While the candidate's claims are impressive on the surface, they lack concrete, verifiable evidence or measurable impact. The descriptions are overly vague and exaggerated, wit...

🤝 INCLUSION POTENTIAL: 7/10
Reasoning: Score: 7/10

Reasoning: While Alex Thompson's technical achievements are undeniably impressive, the information provided does not demonstrate any specific inclusive behaviors or efforts to foster dive...

🔍 BACKGROUND CHECK: PARTIALLY_VERIFIED
Verification Details:
  • verify_university: University 'International University of Advanced Sciences' verification: NOT VERIFIED
  • verify_company: Company 'TechnoFuture Corp' verification: NOT VE

## 🎯 Step 8: Try Your Own Candidate!

Now it's your turn! Paste a candidate's information below and see how our A2A system evaluates them.

In [ ]:
# Try your own candidate here!
your_candidate = """
Name: [Your candidate's name]
Education: [Educational background]
Experience: [Work experience]
Projects: [Notable projects]
Skills: [Technical skills]
Other: [Any other relevant information]
"""

# Uncomment the lines below to evaluate your candidate
# print("Evaluating Your Candidate:")
# your_evaluation = coordinator.evaluate_candidate(your_candidate)
# coordinator.print_evaluation_report(your_evaluation)

## 🚀 Congratulations!

You've successfully built and tested a simple A2A (Agent-to-Agent) hiring assistant! 

### What You Accomplished:
- ✅ Created two specialized AI agents with tool calling
- ✅ Implemented bias-aware evaluation criteria
- ✅ Built agent coordination and communication
- ✅ Tested with realistic candidate scenarios
- ✅ Learned key A2A concepts hands-on

### Ready for More?
This simplified version demonstrates the core concepts. The full workshop includes:
- Advanced multi-agent frameworks (AutoGen)
- Complete A2A protocol implementation
- Web-based user interface
- Enterprise-grade features
- Real-time agent communication

Check out the complete implementation in the `samples/` directory and try the full demo!

---
*Happy coding! 🎉*